In [9]:
import numpy as np
import pandas as pd
import ctypes
from customProphet6 import *

# Load the shared library
lib = ctypes.CDLL('./libgradient.so')

# Define argument and return types
lib.gradient.argtypes = [
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    ctypes.c_double,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS')
]

# Load data
df = pd.read_csv('peyton_manning.csv')

# Instantiate & initialize a model
model = CustomProphet()
model.y = df['y'].values
if df['ds'].dtype != 'datetime64[ns]':
    model.ds = pd.to_datetime(df['ds'])
else:
    model.ds = df['ds']

model.t_scaled = np.array((model.ds - model.ds.min()) / (model.ds.max() - model.ds.min()))
model.T = df.shape[0]

model.scale_period = (model.ds.max() - model.ds.min()).days
model._normalize_y()
model._generate_change_points()

params = np.zeros((47,))

# Convert data to ctypes
params_ctypes = np.ascontiguousarray(params, dtype=np.float64)
t_scaled_ctypes = np.ascontiguousarray(model.t_scaled, dtype=np.float64)
change_points_ctypes = np.ascontiguousarray(model.change_points, dtype=np.float64)
normalized_y_ctypes = np.ascontiguousarray(model.normalized_y, dtype=np.float64)

# Create an array to store the gradient
gradient = np.zeros(params.size, dtype=np.float64)
gradient_ctypes = np.ascontiguousarray(gradient, dtype=np.float64)

# Call the C++ function
lib.gradient(
    params_ctypes, len(params_ctypes),
    t_scaled_ctypes, len(t_scaled_ctypes),
    change_points_ctypes, len(change_points_ctypes),
    model.scale_period,
    normalized_y_ctypes, len(normalized_y_ctypes),
    model.sigma_obs,
    model.sigma_k,
    model.sigma_m,
    model.sigma,
    model.tau,
    gradient_ctypes
)

python_gradient = model._gradient(params)

print("Gradient (C++):", gradient_ctypes)
print("Gradient (Python):", python_gradient)

are_close = np.allclose(gradient_ctypes, python_gradient)
differences = gradient_ctypes - python_gradient

print('Are the gradients approximately equal?', are_close)
print('Differences:', differences)

Gradient (C++): [-6.50281313e+06 -1.28196555e+07 -6.09949595e+06 -5.70847755e+06
 -5.32766111e+06 -4.95851412e+06 -4.60232428e+06 -4.25841944e+06
 -3.92675603e+06 -3.60785480e+06 -3.30277580e+06 -3.01018283e+06
 -2.72941658e+06 -2.46301660e+06 -2.21063757e+06 -1.97092168e+06
 -1.74448750e+06 -1.53223622e+06 -1.33506859e+06 -1.15134391e+06
 -9.81472931e+05 -8.26307575e+05 -6.84855313e+05 -5.55820069e+05
 -4.40338925e+05 -3.39569304e+05 -2.52348659e+05 -7.78835641e+05
 -5.37090972e+04 -4.21551153e+04 -8.72515574e+04 -4.67143626e+04
 -4.12627810e+05 -5.08425390e+03 -9.07461562e+04  1.27479881e+05
  3.98940951e+02  2.91670792e+04 -3.24692556e+03  1.58900888e+03
  5.73131895e+04 -2.74975728e+04 -9.67803754e+04 -4.28063130e+04
  1.08509800e+04  6.59436141e+03  8.83145154e+04]
Gradient (Python): [-6.50281313e+06 -1.28196555e+07 -6.09951595e+06 -5.70849755e+06
 -5.32768111e+06 -4.95853412e+06 -4.60234428e+06 -4.25843944e+06
 -3.92677603e+06 -3.60787480e+06 -3.30279580e+06 -3.01020283e+06
 -2.7

In [5]:
print('Are the gradients approximately equal?', are_close)

Are the gradients approximately equal? False


In [10]:
differences

array([ 0.00000000e+00, -5.58793545e-09,  2.00000000e+01,  2.00000000e+01,
        2.00000000e+01,  2.00000000e+01,  2.00000000e+01,  2.00000000e+01,
        2.00000000e+01,  2.00000000e+01,  2.00000000e+01,  2.00000000e+01,
        2.00000000e+01,  2.00000000e+01,  2.00000000e+01,  2.00000000e+01,
        2.00000000e+01,  2.00000000e+01,  2.00000000e+01,  2.00000000e+01,
        2.00000000e+01,  2.00000000e+01,  2.00000000e+01,  2.00000000e+01,
        2.00000000e+01,  2.00000000e+01,  2.00000000e+01,  9.31322575e-10,
       -1.15539819e+04,  4.55924727e+03, -8.21673035e+04, -1.74194243e+05,
       -4.41794889e+05, -6.67326278e+03, -6.32485833e+04,  1.70286194e+05,
       -6.19542046e+03,  8.28761764e+04,  8.40046319e+04,  4.14216819e+05,
        1.48059346e+05, -2.78965138e+04, -9.35334498e+04, -1.00119502e+05,
        1.07631355e+05, -4.25661860e+03, -8.73114914e-11])

In [11]:
import numpy as np
import pandas as pd
import ctypes
from customProphet6 import *

# Load the shared library
lib = ctypes.CDLL('./libgradient_2.so')

# Define argument and return types
lib.gradient.argtypes = [
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    ctypes.c_double,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS')
]

# Load data
df = pd.read_csv('peyton_manning.csv')

# Instantiate & initialize a model
model = CustomProphet()
model.y = df['y'].values
if df['ds'].dtype != 'datetime64[ns]':
    model.ds = pd.to_datetime(df['ds'])
else:
    model.ds = df['ds']

model.t_scaled = np.array((model.ds - model.ds.min()) / (model.ds.max() - model.ds.min()))
model.T = df.shape[0]

model.scale_period = (model.ds.max() - model.ds.min()).days
model._normalize_y()
model._generate_change_points()

params = np.zeros((47,))

# Convert data to ctypes
params_ctypes = np.ascontiguousarray(params, dtype=np.float64)
t_scaled_ctypes = np.ascontiguousarray(model.t_scaled, dtype=np.float64)
change_points_ctypes = np.ascontiguousarray(model.change_points, dtype=np.float64)
normalized_y_ctypes = np.ascontiguousarray(model.normalized_y, dtype=np.float64)

# Create an array to store the gradient
gradient = np.zeros(params.size, dtype=np.float64)
gradient_ctypes = np.ascontiguousarray(gradient, dtype=np.float64)

# Call the C++ function
lib.gradient(
    params_ctypes, len(params_ctypes),
    t_scaled_ctypes, len(t_scaled_ctypes),
    change_points_ctypes, len(change_points_ctypes),
    model.scale_period,
    normalized_y_ctypes, len(normalized_y_ctypes),
    model.sigma_obs,
    model.sigma_k,
    model.sigma_m,
    model.sigma,
    model.tau,
    gradient_ctypes
)

python_gradient = model._gradient(params)

print("Gradient (C++):", gradient_ctypes)
print("Gradient (Python):", python_gradient)

are_close = np.allclose(gradient_ctypes, python_gradient)
differences = gradient_ctypes - python_gradient

print('Are the gradients approximately equal?', are_close)
print('Differences:', differences)

Gradient (C++): [-1.27411494e+05 -2.51179209e+05 -1.19489576e+05 -1.11828240e+05
 -1.04366793e+05 -9.71339898e+04 -9.01550594e+04 -8.34168326e+04
 -7.69184557e+04 -7.06701320e+04 -6.46926310e+04 -5.89597726e+04
 -5.34586384e+04 -4.82389868e+04 -4.32940516e+04 -3.85972291e+04
 -3.41606388e+04 -3.00019383e+04 -2.61387760e+04 -2.25390053e+04
 -1.92106742e+04 -1.61704746e+04 -1.33989596e+04 -1.08707348e+04
 -8.60807947e+03 -6.63367121e+03 -4.92473195e+03 -1.52599514e+04
 -1.05233784e+03 -8.25957337e+02 -1.70954494e+03 -9.15287984e+02
 -8.08473573e+03 -9.96172536e+01 -1.77801562e+03  2.49775008e+03
  7.81656516e+00  5.71478999e+02 -6.36179492e+01  3.11339095e+01
  1.12295386e+03 -5.38767878e+02 -1.89624582e+03 -8.38716441e+02
  2.12606382e+02  1.29205226e+02  1.73037178e+03]
Gradient (Python): [-1.27411494e+05 -2.51179209e+05 -1.19509576e+05 -1.11848240e+05
 -1.04386793e+05 -9.71539898e+04 -9.01750594e+04 -8.34368326e+04
 -7.69384557e+04 -7.06901320e+04 -6.47126310e+04 -5.89797726e+04
 -5.3

In [12]:
differences

array([ 0.00000000e+00, -1.16415322e-10,  2.00000000e+01,  2.00000000e+01,
        2.00000000e+01,  2.00000000e+01,  2.00000000e+01,  2.00000000e+01,
        2.00000000e+01,  2.00000000e+01,  2.00000000e+01,  2.00000000e+01,
        2.00000000e+01,  2.00000000e+01,  2.00000000e+01,  2.00000000e+01,
        2.00000000e+01,  2.00000000e+01,  2.00000000e+01,  2.00000000e+01,
        2.00000000e+01,  2.00000000e+01,  2.00000000e+01,  2.00000000e+01,
        2.00000000e+01,  2.00000000e+01,  2.00000000e+01,  1.81898940e-11,
       -2.26380501e+02,  8.93306472e+01, -1.60992769e+03, -3.41303807e+03,
       -8.65621473e+03, -1.30751163e+02, -1.23924774e+03,  3.33646652e+03,
       -1.21388661e+02,  1.62381684e+03,  1.64592699e+03,  8.11586964e+03,
        2.90096948e+03, -5.46584444e+02, -1.83262787e+03, -1.96167030e+03,
        2.10885220e+03, -8.34011564e+01, -1.59161573e-12])

In [13]:
gradient_ctypes

array([-1.27411494e+05, -2.51179209e+05, -1.19489576e+05, -1.11828240e+05,
       -1.04366793e+05, -9.71339898e+04, -9.01550594e+04, -8.34168326e+04,
       -7.69184557e+04, -7.06701320e+04, -6.46926310e+04, -5.89597726e+04,
       -5.34586384e+04, -4.82389868e+04, -4.32940516e+04, -3.85972291e+04,
       -3.41606388e+04, -3.00019383e+04, -2.61387760e+04, -2.25390053e+04,
       -1.92106742e+04, -1.61704746e+04, -1.33989596e+04, -1.08707348e+04,
       -8.60807947e+03, -6.63367121e+03, -4.92473195e+03, -1.52599514e+04,
       -1.05233784e+03, -8.25957337e+02, -1.70954494e+03, -9.15287984e+02,
       -8.08473573e+03, -9.96172536e+01, -1.77801562e+03,  2.49775008e+03,
        7.81656516e+00,  5.71478999e+02, -6.36179492e+01,  3.11339095e+01,
        1.12295386e+03, -5.38767878e+02, -1.89624582e+03, -8.38716441e+02,
        2.12606382e+02,  1.29205226e+02,  1.73037178e+03])

In [15]:
python_gradient

array([-1.27411494e+05, -2.51179209e+05, -1.19509576e+05, -1.11848240e+05,
       -1.04386793e+05, -9.71539898e+04, -9.01750594e+04, -8.34368326e+04,
       -7.69384557e+04, -7.06901320e+04, -6.47126310e+04, -5.89797726e+04,
       -5.34786384e+04, -4.82589868e+04, -4.33140516e+04, -3.86172291e+04,
       -3.41806388e+04, -3.00219383e+04, -2.61587760e+04, -2.25590053e+04,
       -1.92306742e+04, -1.61904746e+04, -1.34189596e+04, -1.08907348e+04,
       -8.62807947e+03, -6.65367121e+03, -4.94473195e+03, -1.52599514e+04,
       -8.25957337e+02, -9.15287984e+02, -9.96172536e+01,  2.49775008e+03,
        5.71478999e+02,  3.11339095e+01, -5.38767878e+02, -8.38716441e+02,
        1.29205226e+02, -1.05233784e+03, -1.70954494e+03, -8.08473573e+03,
       -1.77801562e+03,  7.81656516e+00, -6.36179492e+01,  1.12295386e+03,
       -1.89624582e+03,  2.12606382e+02,  1.73037178e+03])

In [1]:
import numpy as np
import pandas as pd
import ctypes
from customProphet6 import *

# Load the shared library
lib = ctypes.CDLL('./libgradient_3.so')

# Define argument and return types
lib.gradient.argtypes = [
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    ctypes.c_double,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS')
]

# Load data
df = pd.read_csv('peyton_manning.csv')

# Instantiate & initialize a model
model = CustomProphet()
model.y = df['y'].values
if df['ds'].dtype != 'datetime64[ns]':
    model.ds = pd.to_datetime(df['ds'])
else:
    model.ds = df['ds']

model.t_scaled = np.array((model.ds - model.ds.min()) / (model.ds.max() - model.ds.min()))
model.T = df.shape[0]

model.scale_period = (model.ds.max() - model.ds.min()).days
model._normalize_y()
model._generate_change_points()

params = np.zeros((47,))

# Convert data to ctypes
params_ctypes = np.ascontiguousarray(params, dtype=np.float64)
t_scaled_ctypes = np.ascontiguousarray(model.t_scaled, dtype=np.float64)
change_points_ctypes = np.ascontiguousarray(model.change_points, dtype=np.float64)
normalized_y_ctypes = np.ascontiguousarray(model.normalized_y, dtype=np.float64)

# Create an array to store the gradient
gradient = np.zeros(params.size, dtype=np.float64)
gradient_ctypes = np.ascontiguousarray(gradient, dtype=np.float64)

# Call the C++ function
lib.gradient(
    params_ctypes, len(params_ctypes),
    t_scaled_ctypes, len(t_scaled_ctypes),
    change_points_ctypes, len(change_points_ctypes),
    model.scale_period,
    normalized_y_ctypes, len(normalized_y_ctypes),
    model.sigma_obs,
    model.sigma_k,
    model.sigma_m,
    model.sigma,
    model.tau,
    gradient_ctypes
)

print("Gradient (C++):", gradient_ctypes)

Gradient (C++): [-8.77513380e+03 -1.72993118e+04 -8.21090984e+03 -7.68325530e+03
 -7.16936769e+03 -6.67122724e+03 -6.19057164e+03 -5.72649387e+03
 -5.27893514e+03 -4.84859817e+03 -4.43691340e+03 -4.04207776e+03
 -3.66320151e+03 -3.30371164e+03 -2.96314215e+03 -2.63966077e+03
 -2.33410220e+03 -2.04768257e+03 -1.78161736e+03 -1.53369256e+03
 -1.05098929e+03 -7.24770195e+01 -5.68856538e+01 -1.17740442e+02
 -6.30380688e+01 -5.56815053e+02 -6.86087810e+00 -1.22456181e+02
  1.72026012e+02  5.38345505e-01  3.93591231e+01 -4.38152006e+00
  2.14426668e+00  7.73405134e+01 -3.71062301e+01 -1.30598977e+02
 -5.77644038e+01  1.46427092e+01  8.89867239e+00  1.19174836e+02
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00]


In [7]:
lib.gradient(
    params_ctypes, len(params_ctypes),
    t_scaled_ctypes, len(t_scaled_ctypes),
    change_points_ctypes, len(change_points_ctypes),
    model.scale_period,
    normalized_y_ctypes, len(normalized_y_ctypes),
    model.sigma_obs,
    model.sigma_k,
    model.sigma_m,
    model.sigma,
    model.tau,
    gradient_ctypes
)
gradient_ctypes
#gradient_ctypes[-20:]

array([-8.77513380e+03, -1.72993118e+04, -8.21090984e+03, -7.68325530e+03,
       -7.16936769e+03, -6.67122724e+03, -6.19057164e+03, -5.72649387e+03,
       -5.27893514e+03, -4.84859817e+03, -4.43691340e+03, -4.04207776e+03,
       -3.66320151e+03, -3.30371164e+03, -2.96314215e+03, -2.63966077e+03,
       -2.33410220e+03, -2.04768257e+03, -1.78161736e+03, -1.53369256e+03,
       -1.05098929e+03, -7.24770195e+01, -5.68856538e+01, -1.17740442e+02,
       -6.30380688e+01, -5.56815053e+02, -6.86087810e+00, -1.22456181e+02,
        1.72026012e+02,  5.38345505e-01,  3.93591231e+01, -4.38152006e+00,
        2.14426668e+00,  7.73405134e+01, -3.71062301e+01, -1.30598977e+02,
       -5.77644038e+01,  1.46427092e+01,  8.89867239e+00,  1.19174836e+02,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00])

In [8]:
gradient_ctypes[-20:]

array([-122.456181  ,  172.02601179,    0.53834551,   39.35912311,
         -4.38152006,    2.14426668,   77.34051342,  -37.10623014,
       -130.59897699,  -57.76440378,   14.64270916,    8.89867239,
        119.17483597,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ])

In [5]:
model._gradient(params)[-20:]

array([-1.05098929e+03, -5.68856538e+01, -6.30380688e+01, -6.86087810e+00,
        1.72026012e+02,  3.93591231e+01,  2.14426668e+00, -3.71062301e+01,
       -5.77644038e+01,  8.89867239e+00, -7.24770195e+01, -1.17740442e+02,
       -5.56815053e+02, -1.22456181e+02,  5.38345505e-01, -4.38152006e+00,
        7.73405134e+01, -1.30598977e+02,  1.46427092e+01,  1.19174836e+02])

In [1]:
import numpy as np
import pandas as pd
import ctypes
from customProphet6 import *

# Load the shared library
lib = ctypes.CDLL('./libgradient_4.so')

# Define argument and return types
lib.gradient.argtypes = [
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    ctypes.c_double,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS')
]

# Load data
df = pd.read_csv('peyton_manning.csv')

# Instantiate & initialize a model
model = CustomProphet()
model.y = df['y'].values
if df['ds'].dtype != 'datetime64[ns]':
    model.ds = pd.to_datetime(df['ds'])
else:
    model.ds = df['ds']

model.t_scaled = np.array((model.ds - model.ds.min()) / (model.ds.max() - model.ds.min()))
model.T = df.shape[0]

model.scale_period = (model.ds.max() - model.ds.min()).days
model._normalize_y()
model._generate_change_points()

params = np.zeros((47,))

# Convert data to ctypes
params_ctypes = np.ascontiguousarray(params, dtype=np.float64)
t_scaled_ctypes = np.ascontiguousarray(model.t_scaled, dtype=np.float64)
change_points_ctypes = np.ascontiguousarray(model.change_points, dtype=np.float64)
normalized_y_ctypes = np.ascontiguousarray(model.normalized_y, dtype=np.float64)

# Create an array to store the gradient
gradient = np.zeros(params.size, dtype=np.float64)
gradient_ctypes = np.ascontiguousarray(gradient, dtype=np.float64)

# Call the C++ function
lib.gradient(
    params_ctypes, len(params_ctypes),
    t_scaled_ctypes, len(t_scaled_ctypes),
    change_points_ctypes, len(change_points_ctypes),
    model.scale_period,
    normalized_y_ctypes, len(normalized_y_ctypes),
    model.sigma_obs,
    model.sigma_k,
    model.sigma_m,
    model.sigma,
    model.tau,
    gradient_ctypes
)

print("Gradient (C++):", gradient_ctypes)

Gradient (C++): [-1.71553691e+05 -3.38201202e+05 -1.60894123e+05 -1.50578488e+05
 -1.40531995e+05 -1.30793359e+05 -1.21396552e+05 -1.12323841e+05
 -1.03574077e+05 -9.51609973e+04 -8.71125683e+04 -7.93935393e+04
 -7.19865162e+04 -6.49584964e+04 -5.83003696e+04 -5.19763151e+04
 -4.60026512e+04 -4.04031530e+04 -3.52015835e+04 -3.03546585e+04
 -2.58732145e+04 -2.17797262e+04 -1.80480092e+04 -1.46438714e+04
 -1.15973104e+04 -8.93886093e+03 -6.63785316e+03 -2.05468197e+04
 -1.41692429e+03 -1.11211340e+03 -2.30182329e+03 -1.23239299e+03
 -1.08857232e+04 -1.34130030e+02 -2.39401590e+03  3.36310511e+03
  1.05246439e+01  7.69470073e+02 -8.56586299e+01  4.19203710e+01
  1.51200550e+03 -7.25426061e+02 -2.55320740e+03 -1.12929294e+03
  2.86264673e+02  1.73968868e+02  2.32986567e+03]


In [2]:
model._gradient(params)

array([-1.71553691e+05, -3.38201202e+05, -1.60914123e+05, -1.50598488e+05,
       -1.40551995e+05, -1.30813359e+05, -1.21416552e+05, -1.12343841e+05,
       -1.03594077e+05, -9.51809973e+04, -8.71325683e+04, -7.94135393e+04,
       -7.20065162e+04, -6.49784964e+04, -5.83203696e+04, -5.19963151e+04,
       -4.60226512e+04, -4.04231530e+04, -3.52215835e+04, -3.03746585e+04,
       -2.58932145e+04, -2.17997262e+04, -1.80680092e+04, -1.46638714e+04,
       -1.16173104e+04, -8.95886093e+03, -6.65785316e+03, -2.05468197e+04,
       -1.11211340e+03, -1.23239299e+03, -1.34130030e+02,  3.36310511e+03,
        7.69470073e+02,  4.19203710e+01, -7.25426061e+02, -1.12929294e+03,
        1.73968868e+02, -1.41692429e+03, -2.30182329e+03, -1.08857232e+04,
       -2.39401590e+03,  1.05246439e+01, -8.56586299e+01,  1.51200550e+03,
       -2.55320740e+03,  2.86264673e+02,  2.32986567e+03])

In [8]:
def loss(model, params):
    k, m, delta, beta = extract_params(params)
    
    A = (model.t_scaled[:, None] > model.change_points) * 1
    gamma = -model.change_points * delta
    g = (k + det_dot(A, delta)) * model.t_scaled + (m + det_dot(A, gamma))

    period = 365.25 / model.scale_period
    x = fourier_components(model.t_scaled, period, 10)
    s = np.dot(x, beta)

    y_pred = g + s
    y_true = model.normalized_y
    r = y_true - y_pred
    
    return np.sum(r**2) / (2 * model.sigma_obs**2) + np.sum(beta**2) / 2*model.sigma**2 \
        + np.sum(np.abs(delta)) / model.tau + k**2 / (2 * model.sigma_k**2) \
        + m**2 / (2 * model.sigma_m**2)
        
def gradient(model, params):
    k, m, delta, beta = extract_params(params)
    
    A = (model.t_scaled[:, None] > model.change_points) * 1
    gamma = -model.change_points * delta
    g = (k + np.dot(A, delta)) * model.t_scaled + (m + np.dot(A, gamma))

    period = 365.25 / model.scale_period
    x = fourier_components(model.t_scaled, period, 10)
    s = np.dot(x, beta)

    y_pred = g + s
    y_true = model.normalized_y
    r = y_true - y_pred
    
    dk = np.array([-np.sum(r * model.t_scaled) / model.sigma_obs**2 + k / model.sigma_k**2])
    dm = np.array([-np.sum(r) / model.sigma_obs**2 + m / model.sigma_m**2])
    ddelta = -np.sum(r[:, None] * (model.t_scaled[:, None] - model.change_points) * A, axis=0) / model.sigma_obs**2 + np.sign(delta) / model.tau
    dbeta = -np.dot(r, x) / model.sigma_obs**2 + beta / model.sigma**2
    
    return np.concatenate([dk, dm, ddelta, dbeta])

def numerical_gradient(loss, model, params, epsilon=1e-6):
    num_grad = np.zeros_like(params)
    for i in range(len(params)):
        params_perturbed = params.copy()
        params_perturbed[i] += epsilon
        loss_plus_epsilon = loss(model, params_perturbed)
        
        params_perturbed[i] -= 2 * epsilon
        loss_minus_epsilon = loss(model, params_perturbed)
        
        num_grad[i] = (loss_plus_epsilon - loss_minus_epsilon) / (2 * epsilon)
    
    return num_grad

## instantiate & initialize a model
model = CustomProphet()
model.y = df['y'].values
if df['ds'].dtype != 'datetime64[ns]':
    model.ds = pd.to_datetime(df['ds'])
else:
    model.ds = df['ds']

model.t_scaled = np.array((model.ds - model.ds.min()) / (model.ds.max() - model.ds.min()))
model.T = df.shape[0]

model.scale_period = (model.ds.max() - model.ds.min()).days
model._normalize_y()
model._generate_change_points()

params = np.zeros((47,))

# Convert data to ctypes
params_ctypes = np.ascontiguousarray(params, dtype=np.float64)
t_scaled_ctypes = np.ascontiguousarray(model.t_scaled, dtype=np.float64)
change_points_ctypes = np.ascontiguousarray(model.change_points, dtype=np.float64)
normalized_y_ctypes = np.ascontiguousarray(model.normalized_y, dtype=np.float64)

# Create an array to store the gradient
gradient = np.zeros(params.size, dtype=np.float64)
gradient_ctypes = np.ascontiguousarray(gradient, dtype=np.float64)

# Call the C++ function
lib.gradient(
    params_ctypes, len(params_ctypes),
    t_scaled_ctypes, len(t_scaled_ctypes),
    change_points_ctypes, len(change_points_ctypes),
    model.scale_period,
    normalized_y_ctypes, len(normalized_y_ctypes),
    model.sigma_obs,
    model.sigma_k,
    model.sigma_m,
    model.sigma,
    model.tau,
    gradient_ctypes
)

numerical_grad = numerical_gradient(loss, model, params)
python_grad = model._gradient(params)
cpp_grad = gradient_ctypes

In [10]:
numerical_grad[-20:] # finite differences

array([-6.90807159e+04, -3.73905021e+03, -4.14344372e+03, -4.50960215e+02,
        1.13071372e+04,  2.58704484e+03,  1.40941003e+02, -2.43896394e+03,
       -3.79680973e+03,  5.84902882e+02, -4.76385863e+03, -7.73898850e+03,
       -3.65990242e+04, -8.04895040e+03,  3.53850482e+01, -2.87993957e+02,
        5.08353233e+03, -8.58417005e+03,  9.62454011e+02,  7.83327012e+03])

In [11]:
gradient_ctypes[-20:] # C++ gradient

array([-2.05468197e+04, -1.41692429e+03, -1.11211340e+03, -2.30182329e+03,
       -1.23239299e+03, -1.08857232e+04, -1.34130030e+02, -2.39401590e+03,
        3.36310511e+03,  1.05246439e+01,  7.69470073e+02, -8.56586299e+01,
        4.19203710e+01,  1.51200550e+03, -7.25426061e+02, -2.55320740e+03,
       -1.12929294e+03,  2.86264673e+02,  1.73968868e+02,  2.32986567e+03])

In [12]:
model._gradient(params)[-20:] # Python method gradient

array([-6.90807159e+04, -3.73905018e+03, -4.14344367e+03, -4.50960229e+02,
        1.13071372e+04,  2.58704482e+03,  1.40940996e+02, -2.43896390e+03,
       -3.79680973e+03,  5.84902877e+02, -4.76385862e+03, -7.73898847e+03,
       -3.65990242e+04, -8.04895038e+03,  3.53850351e+01, -2.87993936e+02,
        5.08353233e+03, -8.58417009e+03,  9.62453986e+02,  7.83327011e+03])

In [57]:
import numpy as np
import pandas as pd
import ctypes
from customProphet6 import *

# Load the shared library
lib = ctypes.CDLL('./libgradient_4.so')

# Define argument and return types
lib.gradient.argtypes = [
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    ctypes.c_double,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS')
]

# Load data
df = pd.read_csv('peyton_manning.csv')

# Instantiate & initialize a model
model = CustomProphet()
model.y = df['y'].values
if df['ds'].dtype != 'datetime64[ns]':
    model.ds = pd.to_datetime(df['ds'])
else:
    model.ds = df['ds']

model.t_scaled = np.array((model.ds - model.ds.min()) / (model.ds.max() - model.ds.min()))
model.T = df.shape[0]

model.scale_period = (model.ds.max() - model.ds.min()).days
model._normalize_y()
model._generate_change_points()

params = np.zeros((47,))

# Convert data to ctypes
params_ctypes = np.ascontiguousarray(params, dtype=np.float64)
t_scaled_ctypes = np.ascontiguousarray(model.t_scaled, dtype=np.float64)
change_points_ctypes = np.ascontiguousarray(model.change_points, dtype=np.float64)
normalized_y_ctypes = np.ascontiguousarray(model.normalized_y, dtype=np.float64)

# Create an array to store the gradient
gradient = np.zeros(params.size, dtype=np.float64)
cpp_grad = np.ascontiguousarray(gradient, dtype=np.float64)

# Call the C++ function
lib.gradient(
    params_ctypes, len(params_ctypes),
    t_scaled_ctypes, len(t_scaled_ctypes),
    change_points_ctypes, len(change_points_ctypes),
    model.scale_period,
    normalized_y_ctypes, len(normalized_y_ctypes),
    model.sigma_obs,
    model.sigma_k,
    model.sigma_m,
    model.sigma,
    model.tau,
    cpp_grad
)

def loss(model, params):
    k, m, delta, beta = extract_params(params)
    
    A = (model.t_scaled[:, None] > model.change_points) * 1
    gamma = -model.change_points * delta
    g = (k + det_dot(A, delta)) * model.t_scaled + (m + det_dot(A, gamma))

    period = 365.25 / model.scale_period
    x = fourier_components(model.t_scaled, period, 10)
    s = np.dot(x, beta)

    y_pred = g + s
    y_true = model.normalized_y
    r = y_true - y_pred
    
    return np.sum(r**2) / (2 * model.sigma_obs**2) + np.sum(beta**2) / 2*model.sigma**2 \
        + np.sum(np.abs(delta)) / model.tau + k**2 / (2 * model.sigma_k**2) \
        + m**2 / (2 * model.sigma_m**2)
        
def gradient(model, params):
    k, m, delta, beta = extract_params(params)
    
    A = (model.t_scaled[:, None] > model.change_points) * 1
    gamma = -model.change_points * delta
    g = (k + np.dot(A, delta)) * model.t_scaled + (m + np.dot(A, gamma))

    period = 365.25 / model.scale_period
    x = fourier_components(model.t_scaled, period, 10)
    s = np.dot(x, beta)

    y_pred = g + s
    y_true = model.normalized_y
    r = y_true - y_pred
    
    dk = np.array([-np.sum(r * model.t_scaled) / model.sigma_obs**2 + k / model.sigma_k**2])
    dm = np.array([-np.sum(r) / model.sigma_obs**2 + m / model.sigma_m**2])
    ddelta = -np.sum(r[:, None] * (model.t_scaled[:, None] - model.change_points) * A, axis=0) / model.sigma_obs**2 + np.sign(delta) / model.tau
    dbeta = -np.dot(r, x) / model.sigma_obs**2 + beta / model.sigma**2
    
    return np.concatenate([dk, dm, ddelta, dbeta])

def numerical_gradient(loss, model, params, epsilon=1e-6):
    num_grad = np.zeros_like(params)
    for i in range(len(params)):
        params_perturbed = params.copy()
        params_perturbed[i] += epsilon
        loss_plus_epsilon = loss(model, params_perturbed)
        
        params_perturbed[i] -= 2 * epsilon
        loss_minus_epsilon = loss(model, params_perturbed)
        
        num_grad[i] = (loss_plus_epsilon - loss_minus_epsilon) / (2 * epsilon)
    
    return num_grad

numerical_grad = numerical_gradient(loss, model, params)
python_grad = model._gradient(params)

In [4]:
numerical_grad # finite differences

array([-1.44045674e+04, -2.83971857e+04, -1.35112123e+04, -1.26450562e+04,
       -1.18014989e+04, -1.09837908e+04, -1.01947845e+04, -9.43299104e+03,
       -8.69831393e+03, -7.99190671e+03, -7.31611747e+03, -6.66798641e+03,
       -6.04605306e+03, -5.45594284e+03, -4.89689083e+03, -4.36588932e+03,
       -3.86430848e+03, -3.39414460e+03, -2.95739294e+03, -2.55041914e+03,
       -2.17413307e+03, -1.83042185e+03, -1.51708690e+03, -1.23125724e+03,
       -9.75451653e+02, -7.52233983e+02, -5.59029038e+02, -1.72522111e+03,
       -9.33790034e+01, -1.03478322e+02, -1.12622765e+01,  2.82384332e+02,
        6.46088320e+01,  3.51985909e+00, -6.09106610e+01, -9.48214883e+01,
        1.46073589e+01, -1.18972558e+02, -1.93273422e+02, -9.14023668e+02,
       -2.01014407e+02,  8.83705980e-01, -7.19235868e+00,  1.26956087e+02,
       -2.14380978e+02,  2.40363170e+01,  1.95628010e+02])

In [5]:
python_grad # Python method gradient

array([-1.44045674e+04, -2.83971857e+04, -1.35112123e+04, -1.26450562e+04,
       -1.18014989e+04, -1.09837908e+04, -1.01947845e+04, -9.43299104e+03,
       -8.69831393e+03, -7.99190670e+03, -7.31611747e+03, -6.66798642e+03,
       -6.04605306e+03, -5.45594285e+03, -4.89689083e+03, -4.36588932e+03,
       -3.86430848e+03, -3.39414460e+03, -2.95739294e+03, -2.55041914e+03,
       -2.17413307e+03, -1.83042186e+03, -1.51708690e+03, -1.23125724e+03,
       -9.75451654e+02, -7.52233983e+02, -5.59029037e+02, -1.72522111e+03,
       -9.33790020e+01, -1.03478321e+02, -1.12622763e+01,  2.82384331e+02,
        6.46088317e+01,  3.51985904e+00, -6.09106603e+01, -9.48214886e+01,
        1.46073586e+01, -1.18972558e+02, -1.93273421e+02, -9.14023667e+02,
       -2.01014407e+02,  8.83705515e-01, -7.19235770e+00,  1.26956086e+02,
       -2.14380979e+02,  2.40363163e+01,  1.95628010e+02])

In [6]:
cpp_grad # C++ gradient

array([-1.44045674e+04, -2.83971857e+04, -1.34912123e+04, -1.26250562e+04,
       -1.17814989e+04, -1.09637908e+04, -1.01747845e+04, -9.41299104e+03,
       -8.67831393e+03, -7.97190670e+03, -7.29611747e+03, -6.64798642e+03,
       -6.02605306e+03, -5.43594285e+03, -4.87689083e+03, -4.34588932e+03,
       -3.84430848e+03, -3.37414460e+03, -2.93739294e+03, -2.53041914e+03,
       -2.15413307e+03, -1.81042186e+03, -1.49708690e+03, -1.21125724e+03,
       -9.55451654e+02, -7.32233983e+02, -5.39029037e+02, -1.72522111e+03,
       -1.18972558e+02, -9.33790020e+01, -1.93273421e+02, -1.03478321e+02,
       -9.14023667e+02, -1.12622763e+01, -2.01014407e+02,  2.82384331e+02,
        8.83705515e-01,  6.46088317e+01, -7.19235770e+00,  3.51985904e+00,
        1.26956086e+02, -6.09106603e+01, -2.14380979e+02, -9.48214886e+01,
        2.40363163e+01,  1.46073586e+01,  1.95628010e+02])

In [7]:
are_close = np.isclose(numerical_grad, python_grad, rtol=1e-1)
are_close


array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

In [8]:
are_close = np.isclose(python_grad[2:27], cpp_grad[2:27], rtol=1e-1) # delta
are_close

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])

In [9]:
are_close = np.isclose(numerical_grad[2:27], cpp_grad[2:27], rtol=1e-1) # delta
are_close

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])

In [10]:
are_close = np.isclose(numerical_grad[-20:], python_grad[-20:], rtol=1e-1)
are_close

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

In [11]:
are_close = np.isclose(python_grad[-20:], cpp_grad[-20:], rtol=1e-1)
are_close

array([ True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True])

In [12]:
are_close = np.isclose(numerical_grad[-20:], cpp_grad[-20:], rtol=1e-1)
are_close

array([ True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True])